In [13]:
# amelioration a faire: changement de type de U
# generalisation du codes (ces codes sont restraintes au cas ou dim_control = 2)
include("./DDP.jl")

using .DDP
using ForwardDiff, DiffResults
using LinearAlgebra

const T = 1
const N = 100
const H = T/N

const D = 4

4

In [ ]:
const X1 = [1, 1, 3/2 * pi, 0]
const P  = [0.1, 0.1, 0.01, 1]

In [1]:
const DIM_x = 4
const DIM_u = 2
const DIM = DIM_x + DIM_u

const C_W = 0.01
const C_A = 0.0001

const WEIGHT = 10000

car = LQR(X1, P, DIM_u, T, N)

"""
F(xu::Vector) -> x::Vector
F(x, u) -> x::Vector
Car dynamics function. xu is the states-controls variable.
h is the time interval. d is the car length.
"""
function F(xu::Vector) 
    
    x, y, theta, v, w, a = xu
#     println("v = ", v)
    f = H*v
#     if (D^2 - f^2 * sin(w)^2) < 0
#         println("v = ", v, "f = ", f, ", f^2 = ", f^2, ", w = ", w, ", sin(w)^2 = ", sin(w)^2)
#     end
    b = f*cos(w) + D - sqrt(D^2 - f^2 * sin(w)^2)

    x = x + b*cos(theta)
    y = y + b*sin(theta)
    theta = theta + asin(sin(w) * f/D)
    v = v + H*a
    
    return [x, y, theta, v]
    
end

function F(x, u) 
    x, y, theta, v = x
    w, a = u
    
    f = H*v
    b = f*cos(w) + D - sqrt(D^2 - f^2 * sin(w)^2)

    x = x + b*cos(theta)
    y = y + b*sin(theta)
    theta = theta + asin(sin(w) * f/D)
    v = v + H*a
    
    return [x, y, theta, v]
    
end

function F(U::Array{Array{Float64,1},1})
    x = X1
    X = [X1]   
    for i in 1:N-1
        x = F(x, U[i])
        push!(X, x)
    end
    return X
end



#---

z(x, p) = sqrt(x^2 + p^2) - p

function L(x_u)
    x, y, theta, v, w, a = x_u
    p_x, p_y, p_theta, p_v = P
    0.01(z(x, p_x) + z(y, p_y)) + C_W * w^2 + C_A * a^2
end

Lf(x) =
    WEIGHT * (z(x[1], P[1]) + z(x[2], P[2]) + z(x[3], P[3]) + z(x[4], P[4]))

Lf (generic function with 1 method)

In [2]:
car = LQR(X1, P, DIM_u, T, N)

LQR([1.0, 1.0, 4.71239, 0.0], [0.1, 0.1, 0.01, 1.0], 4, 2, 6, 1, 100, Main.DDP.Trajectory(Array{Float64,1}[[1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0]  …  [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0]], Array{Float64,1}[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]  …  [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]))

In [10]:
size(car.X1)[1]

4

In [4]:
J, go! = interface(car, F, L, Lf)

(getfield(Main.DDP, Symbol("#J#15")){Int64,getfield(Main.DDP, Symbol("#j#13")){typeof(F),typeof(L),Array{Float64,1}},getfield(Main.DDP, Symbol("#jf#14")){typeof(F),typeof(Lf),Int64,Array{Float64,1}}}(100, getfield(Main.DDP, Symbol("#j#13")){typeof(F),typeof(L),Array{Float64,1}}(F, L, [1.0, 1.0, 4.71239, 0.0]), getfield(Main.DDP, Symbol("#jf#14")){typeof(F),typeof(Lf),Int64,Array{Float64,1}}(F, Lf, 100, [1.0, 1.0, 4.71239, 0.0]), Core.Box(getfield(Main.DDP, Symbol("#J#15")){Int64,getfield(Main.DDP, Symbol("#j#13")){typeof(F),typeof(L),Array{Float64,1}},getfield(Main.DDP, Symbol("#jf#14")){typeof(F),typeof(Lf),Int64,Array{Float64,1}}}(#= circular reference @-2 =#))), getfield(Main.DDP, Symbol("#go!#20")){typeof(F),Int64,Int64,Int64,getfield(Main.DDP, Symbol("#dF#10")){typeof(F)},getfield(Main.DDP, Symbol("#diffs_L#11")){typeof(L)},getfield(Main.DDP, Symbol("#diffs_Lf#12")){typeof(Lf)}}(F, 100, 4, 6, getfield(Main.DDP, Symbol("#dF#10")){typeof(F)}(F), getfield(Main.DDP, Symbol("#diffs_L#1

In [12]:
for _ in 1:100
    go!(car.path)
end

1.52587890625e-5
[1.0, 0.99035, 4.71239, 8.43165e-7]


DomainError: DomainError with -2.903833201538604e21:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [8]:
using ForwardDiff, DiffResults

In [3]:
z(x, p) = sqrt(x^2 + p^2) - p

function L(x_u)
    x, y, theta, v, w, a = x_u
    p_x, p_y, p_theta, p_v = P
    0.01(z(x, p_x) + z(y, p_y)) + C_W * w^2 + C_A * a^2
end

Lf(x) =
    WEIGHT * (z(x[1], P[1]) + z(x[2], P[2]) + z(x[3], P[3]) + z(x[4], P[4]))

Lf (generic function with 1 method)

In [4]:
const T = 1
const N = 100
const H = T/N

const D = 4

const X1 = [1, 1, 3/2 * pi, 0]
const P  = [0.1, 0.1, 0.01, 1]

const DIM_x = 4
const DIM_u = 2
const DIM = DIM_x + DIM_u

const C_W = 0.01
const C_A = 0.0001

const WEIGHT = 1

1

In [5]:
car = LQR(X1, P, DIM_u, T, N)

LQR([1.0, 1.0, 4.71239, 0.0], [0.1, 0.1, 0.01, 1.0], 2, 1, 100, Main.DDP.Trajectory(Array{Float64,1}[[1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0]  …  [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0], [1.0, 1.0, 4.71239, 0.0]], Array{Float64,1}[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]  …  [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]))

In [6]:
J(car.path.U)

UndefVarError: UndefVarError: N not defined